# Getting Started with ghops

Welcome to ghops - a powerful git repository orchestration platform that helps you manage projects across multiple platforms. This notebook will introduce you to the core concepts and basic operations.

## Table of Contents
1. [Introduction to ghops Philosophy](#introduction)
2. [Installation and Setup](#installation)
3. [Understanding the Architecture](#architecture)
4. [Basic Commands and Operations](#basic-commands)
5. [Working with JSONL Output](#jsonl-output)
6. [Configuration](#configuration)
7. [Your First Repository Scan](#first-scan)
8. [Exercises](#exercises)

## 1. Introduction to ghops Philosophy {#introduction}

ghops follows the **Unix Philosophy**:
- **Do one thing well**: Each command has a single, clear purpose
- **Compose via pipes**: Commands can be chained together
- **Text streams are universal**: JSONL (JSON Lines) is our text format

**Key Principle**: Local-first with remote awareness. Your local git repositories are the source of truth, and remote platforms (GitHub, GitLab, PyPI) are services that enrich and distribute your projects.

## 2. Installation and Setup {#installation}

First, let's verify ghops is installed and check its version:

In [6]:
# Check if ghops is installed
import subprocess
import json
import sys

def run_command(cmd):
    """Helper function to run shell commands"""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return result.stdout, result.stderr, result.returncode

# Check ghops version
stdout, stderr, code = run_command("ghops --version")
if code == 0:
    print(f"✓ ghops is installed: {stdout.strip()}")
else:
    print("✗ ghops is not installed. Please install with: pip install ghops")
    print("  Or for development: pip install -e .")

✓ ghops is installed: ghops, version 0.7.0


In [ ]:
# If ghops is not installed, you can install it from within the notebook
# Uncomment the following line to install:
# !pip install ghops

# For development installation (if you have the source code):
# !pip install -e /path/to/ghops

## 3. Understanding the Architecture {#architecture}

ghops is organized in layers:

```
Commands (CLI) → Core Functions → Data Models
     ↓                ↓              ↓
   Parse args    Pure functions   Plain dicts
   Handle I/O    No side effects  Consistent schema
   Format output Return generators Stream-friendly
```

Let's explore the available commands:

In [3]:
# List all available ghops commands
stdout, stderr, code = run_command("ghops --help")
if code == 0:
    # Parse and display main commands
    print("Available ghops commands:")
    print("=" * 50)
    lines = stdout.split('\n')
    in_commands = False
    for line in lines:
        if 'Commands:' in line:
            in_commands = True
            continue
        if in_commands and line.strip() and not line.startswith(' '):
            break
        if in_commands and line.strip():
            print(line)

Available ghops commands:
  ai-cmd       Import and analyze AI conversation data.
  audit        Audit repositories for health and compliance issues.
  catalog      Manage repository catalogs for virtual organization.
  config       Configuration management commands.
  docs         Manage project documentation.
  export       Export repository data in various formats.
  get          Clone repositories from GitHub.
  list         List available repositories with deduplication by default.
  metadata     Manage repository metadata store.
  network-cmd  Analyze and visualize repository relationship networks.
  query        Query repositories using a simple but powerful query...
  service      Automated services for reporting and social media.
  social-cmd   Manage social media posts for repositories.
  status       Show repository status.
  update       Update Git repositories by pulling latest changes.


## 4. Basic Commands and Operations {#basic-commands}

Let's explore some basic ghops commands. We'll start by understanding how to work with repositories.

In [15]:
import os
import tempfile
import shutil
from pathlib import Path


os.chdir(Path.home())
# Create a temporary directory for our examples

# Let's create  a dir in the current working directory instead of a system temp dir
temp_dir = Path.cwd() / "ghops_tutorial_temp"
temp_dir.mkdir(parents=True, exist_ok=True)
print(f"Created temporary directory: {temp_dir}")

# Create some sample git repositories
def create_sample_repo(path, name):
    repo_path = Path(path) / name
    repo_path.mkdir(parents=True, exist_ok=True)
    os.chdir(repo_path)
    
    # Initialize git repo
    run_command("git init")
    run_command("git config user.email 'test@example.com'")
    run_command("git config user.name 'Test User'")
    
    # Create a README
    (repo_path / "README.md").write_text(f"# {name}\n\nSample repository for ghops tutorial.")
    
    # Create initial commit
    run_command("git add .")
    run_command("git commit -m 'Initial commit'")
    
    return repo_path

# Create sample repositories
repo1 = create_sample_repo(temp_dir, "project-alpha")
repo2 = create_sample_repo(temp_dir, "project-beta")
repo3 = create_sample_repo(temp_dir, "project-gamma")

print(f"Created sample repositories:")
print(f"  - {repo1}")
print(f"  - {repo2}")
print(f"  - {repo3}")

Created temporary directory: /home/spinoza/ghops_tutorial_temp
Created sample repositories:
  - /home/spinoza/ghops_tutorial_temp/project-alpha
  - /home/spinoza/ghops_tutorial_temp/project-beta
  - /home/spinoza/ghops_tutorial_temp/project-gamma


In [ ]:
# List repositories in our temporary directory
os.chdir(Path.home())
stdout, stderr, code = run_command(f"ghops list -d {temp_dir}")

print("stdout:", stdout)
print("stderr:", stderr)
print("code:", code)

if code != 0:
    print("⚠️ Unable to list repositories.")
    if stderr:
        print("Error output:")
        print("=" * 50)
        print(stderr)
elif stdout.strip():
    print("Raw JSONL output (first repository):")
    print("=" * 50)
    first_line = stdout.split('\n')[0]
    if first_line:
        print(first_line)
        print()
        print("Parsed JSON:")
        print("=" * 50)
        repo_data = json.loads(first_line)
        print(json.dumps(repo_data, indent=2))
else:
    print("No repositories returned. Verify ghops is configured to scan this directory.")



0
No repositories returned. Verify ghops is configured to scan this directory.


## 5. Working with JSONL Output {#jsonl-output}

JSONL (JSON Lines) is the default output format for ghops. Each line is a valid JSON object, making it perfect for streaming and Unix pipelines.

In [ ]:
# Parse JSONL output into Python objects
def parse_jsonl(output):
    """Parse JSONL output into a list of Python dictionaries"""
    results = []
    for line in output.strip().split('\n'):
        if line:
            try:
                results.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Warning: Could not parse line: {line}")
    return results

# Get repository list and parse it
stdout, _, _ = run_command(f"ghops list {temp_dir}")
repos = parse_jsonl(stdout)

print(f"Found {len(repos)} repositories:")
for repo in repos:
    print(f"  - {repo.get('name', 'unknown')}: {repo.get('path', 'unknown')}")

In [ ]:
# Demonstrate Unix pipeline composition
print("Unix Pipeline Examples:")
print("=" * 50)

# Example 1: Count repositories
stdout, _, _ = run_command(f"ghops list {temp_dir} | wc -l")
print(f"Repository count (using wc): {stdout.strip()}")

# Example 2: Extract just repository names using jq (if available)
stdout, stderr, code = run_command(f"ghops list {temp_dir} | jq -r '.name' 2>/dev/null")
if code == 0:
    print(f"\nRepository names (using jq):")
    print(stdout)
else:
    print("\nNote: jq is not installed. Install it for better JSON processing.")

# Example 3: Using Python for processing
stdout, _, _ = run_command(f"ghops list {temp_dir}")
repos = parse_jsonl(stdout)
names = [r['name'] for r in repos]
print(f"\nRepository names (using Python): {', '.join(names)}")

## 6. Configuration {#configuration}

ghops uses a configuration file to store settings. Let's explore the configuration system:

In [ ]:
# Check current configuration
stdout, stderr, code = run_command("ghops config show")
if code == 0:
    print("Current ghops configuration:")
    print("=" * 50)
    config = json.loads(stdout)
    print(json.dumps(config, indent=2))
else:
    print("No configuration found. Creating default configuration...")
    stdout, _, _ = run_command("ghops config init")
    print(stdout)

In [ ]:
# Set repository directories in configuration
import json
import os

# Get home directory for config path
config_path = Path.home() / ".ghops" / "config.json"
config_path.parent.mkdir(parents=True, exist_ok=True)

# Create or update configuration
config = {
    "general": {
        "repository_directories": [
            str(temp_dir),
            "~/projects/**",  # Example: scan all subdirectories in ~/projects
            "~/github/**"     # Example: scan all subdirectories in ~/github
        ]
    },
    "github": {
        "token": os.environ.get("GHOPS_GITHUB_TOKEN", ""),
        "rate_limit": {
            "max_retries": 3,
            "max_delay_seconds": 60
        }
    }
}

# Save configuration
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print(f"Configuration saved to: {config_path}")
print("\nConfiguration content:")
print(json.dumps(config, indent=2))

## 7. Your First Repository Scan {#first-scan}

Now let's perform a comprehensive repository scan and check status:

In [ ]:
# Get detailed status of repositories
stdout, _, _ = run_command(f"ghops status {temp_dir}")
statuses = parse_jsonl(stdout)

print("Repository Status Report")
print("=" * 50)

for status in statuses:
    print(f"\nRepository: {status.get('name', 'unknown')}")
    print(f"  Path: {status.get('path', 'unknown')}")
    
    if 'status' in status:
        git_status = status['status']
        print(f"  Git Status:")
        print(f"    - Branch: {git_status.get('branch', 'unknown')}")
        print(f"    - Clean: {git_status.get('clean', False)}")
        print(f"    - Has upstream: {git_status.get('has_upstream', False)}")
    
    if 'remote' in status:
        remote = status['remote']
        print(f"  Remote:")
        print(f"    - URL: {remote.get('url', 'none')}")

In [ ]:
# Add some changes to demonstrate status detection
os.chdir(repo1)

# Create a new file
Path("new_feature.py").write_text("# New feature implementation\nprint('Hello, ghops!')")

# Modify README
readme = Path("README.md")
readme.write_text(readme.read_text() + "\n\n## New Section\nThis is a modification.")

# Check status again
stdout, _, _ = run_command(f"ghops status {repo1}")
status = parse_jsonl(stdout)[0]

print("Repository with uncommitted changes:")
print("=" * 50)
print(json.dumps(status, indent=2))

In [ ]:
# Use pretty output for human-readable display
print("Human-readable status (--pretty flag):")
print("=" * 50)
stdout, _, _ = run_command(f"ghops status --pretty {temp_dir}")
print(stdout)

## 8. Exercises {#exercises}

Now it's your turn! Try these exercises to reinforce your learning:

### Exercise 1: Repository Discovery
Find all git repositories in your home directory and count them.

In [ ]:
# TODO: Write code to:
# 1. List all repositories in your home directory
# 2. Count how many you have
# 3. Group them by their parent directory

# Hint: Use ghops list ~/
# Your code here:

### Exercise 2: Status Analysis
Find repositories with uncommitted changes.

In [ ]:
# TODO: Write code to:
# 1. Get status of all repositories
# 2. Filter for repositories with uncommitted changes
# 3. Display a summary

# Your code here:

### Exercise 3: Pipeline Creation
Create a Unix pipeline that finds all Python repositories.

In [ ]:
# TODO: Write a command pipeline that:
# 1. Lists all repositories
# 2. Filters for those with Python files
# 3. Counts them

# Your code here:

## Cleanup

Let's clean up our temporary directory:

In [ ]:
# Clean up temporary directory
if 'temp_dir' in locals() and os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
    print(f"Cleaned up temporary directory: {temp_dir}")

## Summary

In this notebook, you learned:
- The Unix philosophy behind ghops
- How to install and configure ghops
- Basic commands for repository management
- Working with JSONL output format
- Creating Unix pipelines with ghops
- Checking repository status

## Next Steps

Continue with the next notebooks to learn about:
- **Notebook 2**: Repository Clustering and Analysis
- **Notebook 3**: Workflow Orchestration
- **Notebook 4**: Advanced Integrations
- **Notebook 5**: Data Analysis and Visualization

## Resources

- [ghops Documentation](https://github.com/yourusername/ghops)
- [Unix Philosophy](https://en.wikipedia.org/wiki/Unix_philosophy)
- [JSONL Specification](https://jsonlines.org/)
- [jq Tutorial](https://stedolan.github.io/jq/tutorial/)